In [1]:
import logging
from utility import add_project_root_to_path

logging.basicConfig(level=logging.ERROR)

add_project_root_to_path()

In [2]:
# Building experiments configs

from experiments.configs import DATA_SOURCE_BY_ALIAS, DEFAULT_UNINFORMED_USERS_CONFIG
from experiments.experiment import Experiment
from user.informed_user import InformedUser

from fee_algorithm.fixed_fee import FixedFee
from fee_algorithm.discrete_fee_perfect_oracle import DiscreteFeePerfectOracle
from fee_algorithm.based_on_trade_count_fee import BasedOnTradeCountFee
from fee_algorithm.adaptive_fee_based_on_block_price_move import AdaptiveBasedOnPreviousBlockPriceMoveFee
from experiments.run_multiple_experiments import run_multiple_experiments, get_experiment_key
from fee_algorithm.dynamic_fee_amm_for_amm import AMMforAMMfee
import pandas as pd

from copy import deepcopy

fee_algos_to_consider = {
    "fixed_fee": FixedFee(exchange_fee_rate=0.003), # 30 bps
    "based_on_trade_count_fee": BasedOnTradeCountFee(a_to_b_exchange_fee_rate=0.003, b_to_a_exchange_fee_rate=0.003), # 30 bps
    "adaptive_based_on_previous_block_price_move_fee": AdaptiveBasedOnPreviousBlockPriceMoveFee(a_to_b_exchange_fee_rate=0.003, b_to_a_exchange_fee_rate=0.003), # 30 bps
    "discrete_fee_perfect_oracle": DiscreteFeePerfectOracle(fee_rate_in_arbitrage_direction=0.0045, fee_rate_in_non_arbitrage_direction=0.0015), # 45/15 bps
    "AMM_fee": AMMforAMMfee(a_to_b_exchange_fee_rate=0.003, b_to_a_exchange_fee_rate=0.003),
}

In [3]:
df_stable_not_stable = pd.read_csv('df_stable_not_stable.csv')
df_stable_stable = pd.read_csv('df_stable_stable.csv')
df_not_stable_not_stable = pd.read_csv('df_not_stable_not_stable.csv')

In [4]:
df_not_stable_not_stable.head()

,Unnamed: 0,Period,Pair,Slope,Std_dev,start_time,end_time,Market_type,A_symbol,B_symbol
0,0,2024-05-06/2024-05-12,ETH/BTC,-0.000134,0.002148,2024-05-06,2024-05-12 23:59:59.999999999,calm_market,ETH,BTC
1,1,2024-05-13/2024-05-19,ETH/BTC,-0.000133,0.002753,2024-05-13,2024-05-19 23:59:59.999999999,calm_market,ETH,BTC
2,2,2024-05-20/2024-05-26,ETH/BTC,0.000825,0.008224,2024-05-20,2024-05-26 23:59:59.999999999,bull_market,ETH,BTC
3,3,2024-05-27/2024-06-02,ETH/BTC,-0.000088,0.002613,2024-05-27,2024-06-02 23:59:59.999999999,calm_market,ETH,BTC
4,4,2024-06-03/2024-06-09,ETH/BTC,-0.000237,0.002042,2024-06-03,2024-06-09 23:59:59.999999999,calm_market,ETH,BTC


In [5]:
df_not_stable_not_stable['start_time'] = pd.to_datetime(df_not_stable_not_stable['start_time'])
df_not_stable_not_stable['end_time'] = pd.to_datetime(df_not_stable_not_stable['end_time'])

df_stable_not_stable['start_time'] = pd.to_datetime(df_stable_not_stable['start_time'])
df_stable_not_stable['end_time'] = pd.to_datetime(df_stable_not_stable['end_time'])

df_stable_stable['start_time'] = pd.to_datetime(df_stable_stable['start_time'])
df_stable_stable['end_time'] = pd.to_datetime(df_stable_stable['end_time'])

In [6]:
df_stable_not_stable['pair_description'] = 'stable_not_stable'
df_stable_stable['pair_description'] = 'stable_stable'
df_not_stable_not_stable['pair_description'] = 'not_stable_not_stable'

df_all_pairs = pd.concat(
    [
    df_stable_not_stable, 
    df_stable_stable, 
    df_not_stable_not_stable
    ], 
    ignore_index=True,
)

In [7]:
df_all_pairs

,Unnamed: 0,Period,Pair,Slope,Std_dev,start_time,end_time,Market_type,A_symbol,B_symbol,pair_description
0,0,2024-05-06/2024-05-12,BTC/USDT,-0.000322,0.004843,2024-05-06,2024-05-12 23:59:59.999999999,bear_market,BTC,USDT,stable_not_stable
1,1,2024-05-13/2024-05-19,BTC/USDT,0.000603,0.004022,2024-05-13,2024-05-19 23:59:59.999999999,bull_market,BTC,USDT,stable_not_stable
2,2,2024-05-20/2024-05-26,BTC/USDT,-0.000025,0.004696,2024-05-20,2024-05-26 23:59:59.999999999,calm_market,BTC,USDT,stable_not_stable
3,3,2024-05-27/2024-06-02,BTC/USDT,-0.000106,0.003576,2024-05-27,2024-06-02 23:59:59.999999999,calm_market,BTC,USDT,stable_not_stable
4,4,2024-06-03/2024-06-09,BTC/USDT,0.000017,0.003479,2024-06-03,2024-06-09 23:59:59.999999999,calm_market,BTC,USDT,stable_not_stable
...,...,...,...,...,...,...,...,...,...,...,...
1164,472,2025-04-07/2025-04-13,AVAX/ETH,0.001201,0.008207,2025-04-07,2025-04-13 23:59:59.999999999,bull_market,AVAX,ETH,not_stable_not_stable
1165,473,2025-04-14/2025-04-20,AVAX/ETH,0.000035,0.004213,2025-04-14,2025-04-20 23:59:59.999999999,calm_market,AVAX,ETH,not_stable_not_stable
1166,474,2025-04-21/2025-04-27,AVAX/ETH,-0.000155,0.005002,2025-04-21,2025-04-27 23:59:59.999999999,calm_market,AVAX,ETH,not_stable_not_stable
1167,475,2025-04-28/2025-05-04,AVAX/ETH,-0.000674,0.004339,2025-04-28,2025-05-04 23:59:59.999999999,bear_market,AVAX,ETH,not_stable_not_stable


In [8]:
print(f"Number of segments to save: {df_all_pairs[["Period", "Pair"]].drop_duplicates().shape[0]}")

Number of segments to save: 1169


### Stable/Non-Stable Pairs

In [9]:
from experiments.experiment import HistoricalDataDescription

experiment_configs = {}

for i, row in df_all_pairs.iterrows():
    data_source = HistoricalDataDescription(
        start_time=row['start_time'],
        end_time=row['end_time'], 
        A_symbol=row['A_symbol'], 
        B_symbol=row['B_symbol'],
        cache_data=True,
    )
    for fee_algo_alias, fee_algo in fee_algos_to_consider.items():
        # if (len(experiment_configs) > 200):
        #     break
        experiment_configs[
            get_experiment_key(
                {
                    "period_alias": str(row['Market_type']),
                    "fee_algorithm": fee_algo_alias,
                    "pair_description": str(row["pair_description"]),
                    "A_symbol": str(row["A_symbol"]),
                    "B_symbol": str(row["B_symbol"]),
                    "start_time": str(row["start_time"]),
                    "end_time": str(row["end_time"]),
                }
            )
        ] = Experiment(
            data=data_source,
            fee_algorithm=deepcopy(fee_algo),
            uninformed_users=deepcopy(DEFAULT_UNINFORMED_USERS_CONFIG),
            informed_user=InformedUser(),
        )

In [10]:
from pprint import pprint

first_key = list(experiment_configs.keys())[0]
pprint(experiment_configs[first_key])

Experiment(data=HistoricalDataDescription(start_time=Timestamp('2024-05-06 00:00:00'),
                                          end_time=Timestamp('2024-05-12 23:59:59.999999999'),
                                          A_symbol='BTC',
                                          B_symbol='USDT',
                                          stable_coin_symbol='USDT',
                                          candle_interval='1m',
                                          cache_data=True),
           fee_algorithm=FixedFee(exchange_fee_rate=0.003),
           informed_user=InformedUser(),
           uninformed_users=UninformedUsersConfig(uninformed_user=UninformedUser(mu=0.00016,
                                                                                 sigma=1e-05),
                                                  probability_of_trade=0.5,
                                                  n_users=1),
           initial_pool_value=25000000,
           network_fee=5,
           rand

In [11]:
print(
    f"Total experiments to run: {len(experiment_configs)}"
)

Total experiments to run: 5845


In [12]:
experiment_results = run_multiple_experiments(
    experiments=experiment_configs,
    return_intermediate_results=False,
    parallel=True,
    max_workers=8, 
    # parallel=False,
)

Running Experiments: 100%|██████████| 5845/5845 [07:54<00:00, 12.32it/s]


In [13]:
from visualizations.compare_fee_algoritms import get_experiments_summary_by_description

combined_df = get_experiments_summary_by_description(
    experiment_results,
).sort_values(by=["period_alias", "fee_algorithm", "pair_description"])

display(combined_df)

,A_symbol,B_symbol,pair_description,period_alias,fee_algorithm,end_time,start_time,iu_markout,iu_trade_count,iu_yield,uu_markout,uu_trade_count,uu_yield,lp_markout,lp_yield,impermanent_loss
3504,ETH,BTC,not_stable_not_stable,bear_market,AMM_fee,2024-07-07 23:59:59.999999999,2024-07-01 00:00:00,3218.32,598,5.27,-55602.67,5099,-58.41,23899.35,15.29,-22401.67
3519,ETH,BTC,not_stable_not_stable,bear_market,AMM_fee,2024-07-28 23:59:59.999999999,2024-07-22 00:00:00,4262.89,643,6.11,-55547.37,5052,-57.44,22809.48,13.70,-10569.98
3569,ETH,BTC,not_stable_not_stable,bear_market,AMM_fee,2024-10-06 23:59:59.999999999,2024-09-30 00:00:00,2208.23,536,4.15,-55770.55,5071,-58.93,25527.32,17.27,-24571.55
3584,ETH,BTC,not_stable_not_stable,bear_market,AMM_fee,2024-10-27 23:59:59.999999999,2024-10-21 00:00:00,2176.79,484,4.48,-56121.96,5083,-58.18,26110.17,18.00,-12184.94
3599,ETH,BTC,not_stable_not_stable,bear_market,AMM_fee,2024-11-17 23:59:59.999999999,2024-11-11 00:00:00,10902.55,1078,7.81,-57745.80,5046,-54.87,16223.25,6.63,36340.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2604,ADA,USDC,stable_not_stable,volatile_market,fixed_fee,2025-03-16 23:59:59.999999999,2025-03-10 00:00:00,24801.83,1061,14.85,-56184.84,5001,-56.37,1073.01,0.40,-77597.51
2610,ADA,USDC,stable_not_stable,volatile_market,fixed_fee,2025-03-23 23:59:59.999999999,2025-03-17 00:00:00,5910.46,522,8.95,-57179.90,5070,-56.26,23309.44,13.90,-50087.61
2630,ADA,USDC,stable_not_stable,volatile_market,fixed_fee,2025-04-20 23:59:59.999999999,2025-04-14 00:00:00,5141.71,506,8.13,-57231.41,5063,-57.38,24244.70,14.88,-46367.15
2640,ADA,USDC,stable_not_stable,volatile_market,fixed_fee,2025-05-04 23:59:59.999999999,2025-04-28 00:00:00,5171.06,630,6.78,-58627.50,5098,-57.84,24816.43,13.97,-47679.26
